In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch.optim as optim

#On importe les packets nécéssaires pour construire notre modèle

On crée une couche de convolution invariante par rotation avec son Backward

In [ ]:
class convZ(torch.autograd.Function):

    # On utilise ici des @staticmethods afin de pouvoir choisir comment le Backward est calculé
    @staticmethod
    # bias est un argument optionnel
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        k=weight.detach().numpy()
        Kernel= torch.tensor([[[[k[0][0][0][2],k[0][0][0][1],k[0][0][0][2]], #On construit notre kernel invariant par rotation.
                    [k[0][0][0][1],k[0][0][0][0],k[0][0][0][1]],
                    [k[0][0][0][2],k[0][0][0][1],k[0][0][0][2]]]]])
        ctx.conv1=nn.Conv2d(1,1,3)
        ctx.conv1.weight=torch.nn.parameter.Parameter(Kernel) #On configure notre couche de convolution avec
                                                              #notre kernel invariant par rotation
        output=ctx.conv1(input)
        
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # Cette fonction n'a qu'une seule sortie, elle ne reçoit donc qu'un seul gradient
    
    @staticmethod
    def backward(ctx, grad_output):
        # On utiliser le gradient calculé par l'autograd pour le couches suivantes pour déterminer
        # le gradient de cette couche convolutionnelle.
        
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None
        k0=torch.tensor([[[[0.,  0., 0.],    #k0,k1,k2 sont les filtres qui correspondent 
                                             # aux différents coefficients du kernel.
                 [ 0.,  1.,  0.],
                 [0.,  0., 0.]]]])
        k2=torch.tensor([[[[1.,  0., 1.],
                 [ 0.,  0.,  0.],
                 [1.,  0., 1.]]]])
        k1=torch.tensor([[[[0.,  1., 0.],
                 [ 1.,  0.,  1.],
                 [0.,  1., 0.]]]])
        C1=F.conv2d(input,k0)   #On calcule les différents Ci évoqués dans la fiche détails du Backward qui correspondent à 
                                #des tensors de dérivées partielles de la sortie du réseau de convolution 
                                #du forward par rapport aux paramètres de notre kernel invriant par rotation.
        C2=F.conv2d(input,k1)
        C3=F.conv2d(input,k2)

        if ctx.needs_input_grad[0]:
            grad_input = None
        if ctx.needs_input_grad[1]:
            
            g=F.conv2d(C1,grad_output) # On calcule les dérivées partielles de la fonction coût par rapport 
                                       #aux 3 degrés de liberté de notre kernel invariant par rotation.
            h=F.conv2d(C2,grad_output)
            i=F.conv2d(C3,grad_output)
            
            grad_weight = torch.cat((g, h, i), 3) # Pour déterminer le gradient final, on concatène les dérivées partielles 
                                                  #de la fonction coût par rapport aux 3 degrés de 
                                                  #liberté de notre kernel invariant par rotation.
        
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias

In [ ]:
class MyConvZ(nn.Module):
    def __init__(self):
        super(MyConvZ, self).__init__()
        self.fn = convZ.apply
        self.weight = nn.Parameter(torch.randn(1, 1, 1, 3)) # On initialise les 3 poids du kernel invariant par 
                                                            # rotation de notre couche convolutionnelle. 
        

    def forward(self, x):
        x = self.fn(x, self.weight)
        return x


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()  #On crée un petit modèle afin de tester notre Backward.
        self.convZ = MyConvZ()
        self.pool = nn.AvgPool2d((1, 3))
        self.fc1 = nn.Linear(3, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.convZ(x)))
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = x.view(2)
        return x

In [ ]:
if __name__ == '__main__':

#On pose construit input1 et labels1 simplement pour que le modèle puisse s'entrainer
#et que l'on puisse tester le Backward, sans que cela ait de réelle signification.

    input1 = torch.Tensor([
                            [[[1, 0, 0, 1,1],
                             [0, 1, 0, 1,0],
                             [0, 0, 0, 1,1],
                             [1, 1, 0, 1,1],
                             [1, 0, 1, 1,1]]],
                             [[[0, 1, 1, 0,0],
                             [1, 0, 1, 0,1],
                             [1, 1, 1, 0,0],
                             [0, 0, 1, 0,0],
                             [0, 1, 0, 0,0]]]
                          ])

    label1 = torch.LongTensor([0., 1.])

    net = Net()

    # Définir une fonction de perte et un optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(input1)
    loss = criterion(outputs, label1)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss = 0.0
    running_loss += loss.item()

    print(running_loss)

print('Finished Training')